In [1]:
import pandas as pd
import numpy as np

In [12]:
col = pd.read_csv('/Users/joamacha/Library/CloudStorage/OneDrive-TexasTechUniversity/Personal/Projects/Code/GitHub/ColonialLegacy/COLDAT_colonies.csv')
efw = pd.read_excel('/Users/joamacha/Library/CloudStorage/OneDrive-TexasTechUniversity/Personal/Projects/Code/GitHub/ColonialLegacy/efotw-2022-master-index-data-for-researchers-iso.xlsx')
vdem = pd.read_excel('/Users/joamacha/Library/CloudStorage/OneDrive-TexasTechUniversity/Personal/Projects/Code/GitHub/ColonialLegacy/VDem.xls')
vdem.rename(columns={'v2x_libdem':'libdem','v2xcl_prpty':'prop_rights','v2clstown':'stateown'}, inplace=True)

In [126]:
#Keep only colonies 
col = col.loc[col['time_total'] > 0]

In [128]:
# Dummy for approximations of State Ownership for colonies beggining 1789
for i in range(len(col)):
    if col.iloc[i,10:18].min() < 1789:
        col.iloc[i,37] = 1
    else:
        pass    
        


In [154]:
# Initialize columns with placeholder NaN
for colonizer in colonizers: 
    col[f'efw_{colonizer}'] = np.nan

In [218]:
#Names in V-Dem Data
cold = {'belgium':'Belgium', 'britain':'United Kingdom', 'france':'France','germany':'Germany','italy':'Italy','netherlands':'Netherlands','portugal':'Portugal','spain':'Spain'}

# Initialize column
col['avg_efw'] = np.nan

for country in col.loc[col['independence'] >= 1789, 'country_code']:
    for colonizer in colonizers: 
        end_year = col.loc[col['country_code']==country, f'colend_{colonizer}'].values[0]
        if end_year < 1789:
            avg_efw = vdem.loc[(vdem['country_name'] == cold[colonizer]) & (vdem['year'] == 1789), 'stateown'].values[0]
            timec = col.loc[col['country_code']==country, f'time_{colonizer}'].values[0]
            col.loc[col['country_code']==country, f'efw_{colonizer}'] = avg_efw * timec
        else: 
            begin_year = col.loc[col['country_code']==country, f'colstart_{colonizer}'].values[0]
            avg_efw = vdem.loc[(vdem['country_name'] == cold[colonizer]) & (vdem['year'] >= begin_year) & (vdem['year'] <= end_year), 'stateown'].mean() 
            timec = col.loc[col['country_code']==country, f'time_{colonizer}'].values[0]
            col.loc[col['country_code']==country, f'efw_{colonizer}'] = avg_efw * timec

col['avg_efw'] = col[col.columns[38:46]].sum(axis=1) / col['time_total']  

In [3]:
#Exclude 2020 because of the pandemic
efw = efw.loc[(efw['Year']<2020) & (efw['Year'] >1999)].rename(columns={'World Bank Region': "wb_region"})
efw.sort_values(by='Countries').head()

,Year,ISO_Code_2,wb_region,Countries,ISO_Code_3,efw,Area 1,Area 2,Area 3,Area 4,Area 5,std
165,2019,AL,Europe & Central Asia,Albania,ALB,7.789804,7.914268,5.258761,9.826940,8.230911,7.700060,1.641201
2970,2002,AL,Europe & Central Asia,Albania,ALB,6.742276,8.223671,4.845155,7.101343,6.097055,7.291095,1.287959
1485,2011,AL,Europe & Central Asia,Albania,ALB,7.520319,8.071567,5.132500,9.641108,6.996151,7.729106,1.644798
1980,2008,AL,Europe & Central Asia,Albania,ALB,7.368699,8.225602,5.479977,9.433083,6.533912,7.121853,1.526479
330,2018,AL,Europe & Central Asia,Albania,ALB,7.759843,7.811582,5.221942,9.760704,8.221900,7.767841,1.632527


In [26]:
means = efw.groupby('ISO_Code_3')['efw','Area 1','Area 2','Area 3','Area 4','Area 5','std'].mean()
stds = efw.groupby('ISO_Code_3')['efw','Area 1','Area 2','Area 3','Area 4','Area 5','std'].std()
last = efw.loc[efw['Year']==2019]
last

<ipython-input-26-a5ae555899df>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  means = efw.groupby('ISO_Code_3')['efw','Area 1','Area 2','Area 3','Area 4','Area 5','std'].mean()
<ipython-input-26-a5ae555899df>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  stds = efw.groupby('ISO_Code_3')['efw','Area 1','Area 2','Area 3','Area 4','Area 5','std'].std()


,Year,ISO_Code_2,World Bank Region,Countries,ISO_Code_3,efw,Area 1,Area 2,Area 3,Area 4,Area 5,std
165,2019,AL,Europe & Central Asia,Albania,ALB,7.789804,7.914268,5.258761,9.826940,8.230911,7.700060,1.641201
166,2019,DZ,Middle East & North Africa,Algeria,DZA,4.860507,4.493283,4.142183,7.287544,2.552219,5.840058,1.790882
167,2019,AO,Sub-Saharan Africa,Angola,AGO,5.550608,8.052308,3.667991,4.667936,5.406168,5.975078,1.641862
168,2019,AR,Latin America & the Caribbean,Argentina,ARG,5.438832,6.547787,4.797545,3.880439,5.966232,5.993349,1.078801
169,2019,AM,Europe & Central Asia,Armenia,ARM,7.979822,7.770128,6.250483,9.519780,8.524505,7.819692,1.196697
...,...,...,...,...,...,...,...,...,...,...,...,...
325,2019,VE,Latin America & the Caribbean,"Venezuela, RB",VEN,2.871351,4.338019,2.441900,0.944935,4.090400,2.517155,1.383658
326,2019,VN,East Asia & Pacific,Vietnam,VNM,6.400031,6.955223,4.983864,6.892635,6.183881,6.983588,0.857859
327,2019,YE,Middle East & North Africa,"Yemen, Rep.",YEM,5.719912,8.267794,2.744222,8.726899,5.377539,3.502867,2.712049
328,2019,ZM,Sub-Saharan Africa,Zambia,ZMB,6.949406,6.578045,5.014693,9.028393,7.691995,6.454841,1.499930


In [222]:
df = pd.merge(col, means, left_on='country_code', right_on='ISO_Code_3', how='left')
df = pd.merge(col, stds, left_on='country_code', right_on='ISO_Code_3', how='left', suffixes=('',"_std"))

In [ ]:
df = pd.read_excel()

In [9]:
df.to_excel('colonies.xlsx')

In [27]:
df = pd.read_excel('/Users/joamacha/Library/CloudStorage/OneDrive-TexasTechUniversity/Personal/Projects/Code/GitHub/ColonialLegacy/colonies_base.xlsx')

In [28]:
# Initiate Columns with nan
df['efw_year'] = np.nan
df['efw_indep'] = np.nan

for country in df['country_code']:
    if country in efw['ISO_Code_3'].unique():
        year = df.loc[df['country_code']==country, 'independence'].values[0] #Extracts independence year
        efw_year = efw.loc[(efw['ISO_Code_3']==country) & (efw['Year']>=year) 
                                        & (efw['efw'].notnull()), 'Year'].values.min() #Earliest available year for EFW
        df.loc[df['country_code']==country, 'efw_year'] = efw_year
        df.loc[df['country_code']==country, 'efw_indep'] = efw.loc[(efw['ISO_Code_3'] == country) & (efw['Year'] == efw_year), 'efw'].values[0]
    else: 
        pass

In [29]:
df = pd.merge(df, means, left_on='country_code', right_on='ISO_Code_3', how='left', suffixes=('',"_avg"))
df = pd.merge(df, stds, left_on='country_code', right_on='ISO_Code_3', how='left', suffixes=('',"_std"))
df = pd.merge(df, last, left_on='country_code', right_on='ISO_Code_3', how='left', suffixes=('',"_2019"))

In [31]:
df.to_excel('colonies.xlsx')

# WORLD VALUE SURVEY

In [8]:
wvs = pd.read_excel('/Users/joamacha/Library/CloudStorage/OneDrive-TexasTechUniversity/Personal/Projects/Code/GitHub/ColonialLegacy/WVS.xls')
df = pd.read_excel('/Users/joamacha/Library/CloudStorage/OneDrive-TexasTechUniversity/Personal/Projects/Code/GitHub/ColonialLegacy/colonies.xlsx')

In [9]:
df = pd.merge(df, wvs, left_on='country_code', right_on='COUNTRY_ALPHA', how='left')
df.to_excel('colonies_wvs.xlsx')